In [ ]:
# %% [markdown]
# # Submission

# %% [markdown]
# ## load package

# %% [code]
import sys
import pandas as pd

from keras.applications import xception
from keras.applications import resnet50
from keras.applications import mobilenetv2
# from efficientnet import EfficientNetB3

import time
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import random

import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense, Softmax
from keras.preprocessing import image

from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, K
from keras.models import Input, Model
from sklearn.metrics import f1_score

from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import gc


# %% [code]
bKaggle = True
bDebug = False   # train data test.

datadir = './pre/'
modeldir = './pre/'
inputdir='./'
if bKaggle:
    datadir = '../input/carmodel5/ppcarmodel/'
    modeldir = '../input/carmodel6/model/'
    inputdir = '../input/2019-3rd-ml-month-with-kakr/'

print(modeldir)

# %% [code]
# 224/299
imagesize=250
# get_ipython().run_line_magic('matplotlib', 'inline')

def dbgprint(msg):
    if bKaggle:
        os.system('echo "'+msg+'"')
    else:
        print(msg)

dbgprint('hello log')

# %% [code]
# f1 score 
def new_score(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = (true_positives + K.epsilon()) / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = (true_positives + K.epsilon()) / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# %% [markdown]
# ### Predict

# %% [code]
# test data load for submission

if bDebug:
    print('debug...')
    dftrain = pd.read_csv(inputdir+'train.csv')
    dftrain['class'] = dftrain['class'].astype('str')
    dftest = dftrain.iloc[8000:,:]
    targetdir = datadir+'train'
else:
    dftest = pd.read_csv(inputdir+'test.csv')
    targetdir = datadir+'test'

classes = list(str(num) for num in range(1,197))

batch_size=32
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_dataframe(dataframe=dftest, 
                                             directory=targetdir,
                                             x_col = 'img_file', 
                                             class_mode = None,
                                             target_size = (imagesize, imagesize),
                                             batch_size=batch_size, shuffle=False )


# %% [code]
if True:
    import glob, os
    
    predictions=[]
    # 'carmodel-v8-1-', 
#     for ff, mp in enumerate(['carmodel-v8-6-', 'carmodel-v8-7-', 'carmodel-v8-8-']):
    for ff, mp in enumerate(['carmodel-v5-1', 'carmodel-v5-2-']):
        files = glob.glob(modeldir+mp+'*')
        mp = max(files, key=os.path.getctime)
        msg = '{} model={}'.format(ff, mp)
        dbgprint(msg)
        
        method='xception'
        if mp.find('resnet50')>0:
            method = 'resnet50'
        elif mp.find('mobilenetv2')>0:
            method = 'mobilenetv2'
        elif mp.find('efficientnetb3')>0:
            method = 'efficientnetb3'
        
        inputs = Input(shape=(imagesize,imagesize,3))
        print('method=', method)
        if method=='xception':
            net = xception.Xception(input_tensor=inputs, input_shape=(imagesize, imagesize, 3), include_top=False, weights='imagenet', pooling='max')
        elif method=='resnet50':
            net = resnet50.ResNet50(input_tensor=inputs, input_shape=(imagesize, imagesize, 3), include_top=False, weights='imagenet', pooling='max')
        elif method=='mobilenetv2':
            net = mobilenetv2.MobileNetV2(input_tensor=inputs, input_shape=(imagesize, imagesize, 3), include_top=False, weights='imagenet', pooling='max')
        elif method=='efficientnetb3':
            net = EfficientNetB3(input_tensor=inputs, input_shape=(imagesize, imagesize, 3), include_top=False, weights='imagenet', pooling='max')
        net2 = Dense(256, activation='relu') (net.layers[-1].output)
        net2 = Dense(196, activation='softmax')(net2)
        model = Model(inputs=inputs, outputs=net2)
        
        print('model',ff,':', mp)
        model.load_weights(mp)
        
        print('predict_generator')
        test_generator.reset()
        pr = model.predict_generator( test_generator , steps=len(dftest)/batch_size, verbose=1)
        # steps=len(dftest)/batchsize
        predictions.append(pr)
        print('prediction',ff,':',pr)
        print(pr.shape)        
        

# %% [code]
print('dftest count=', len(dftest))
print('model result count=', len(predictions))
print('each result count=', predictions[0].shape)

# %% [code]
# �덉륫留� 諛섎났
if False:
    predictions=[]
    test_generator.reset()
    pr = model.predict_generator( test_generator , steps=len(dftest)/batch_size, verbose=1 )
    # steps=len(dftest)/batchsize
    predictions.append(pr)
    print('prediction',ff,':',pr)
    print(pr.shape)


# %% [code]
print(np.argmax(predictions[0], axis=1))
print(np.argmax(predictions[1], axis=1))

# %% [code]
if True:
    predictions = np.asarray(predictions)
    prk = np.mean(predictions, axis=0 )
    pdi = np.argmax(prk, axis=1)
    print('middle:', pdi, np.min(pdi), np.max(pdi))
    
    sns.countplot(pdi)


# %% [code]
if True:
    # labeling k=label string, v=class number
#     label = dict((v,k) for k,v in train_generator.class_indices.items())
#     final_pred = [label[k] for k in pdi]
    
    n1=pd.DataFrame(list(range(1,197)), columns=['num']).astype('str')
    n2=n1.sort_values('num').values.squeeze()
    final_pred = [n2[k] for k in pdi]
    
    final_pred = np.asarray(final_pred, dtype=np.int)
    print('final:', final_pred)
    print('count=', len(final_pred))    

# %% [code]
if bDebug:
    reallabel = np.array(dftest['class'].values, dtype=int)
    print(reallabel)
    print( np.sum(final_pred==reallabel), '/', len(final_pred), len(reallabel))

# %% [code] {"jupyter":{"outputs_hidden":true}}
if not bDebug:
    submission = pd.read_csv(inputdir+'sample_submission.csv')
    submission["class"] = final_pred  
    submission.to_csv("submission.csv", index=False)
    submission.head()

    sns.countplot(submission["class"], order=submission["class"].value_counts(ascending=True).index)

# %% [code]
if False:
    dfpredictions = pd.DataFrame(np.max(final_pred, axis=1).squeeze() )
    dfpredictions.describe()
    dfpredictions[0].plot.hist(bins=20)
    psids = dfpredictions[0]>0.99 
    np.sum(psids)

# %% [code]
# pseudo dataset
# x_test[psids]
# pdi[psids]
# make pseudo
if False:
    print('Create Pseudo Label')
    x_test = np.load(datadir+'x_test3.npy')
    np.save('x_pseudo3.npy', x_test[psids])
    np.save('y_pseudo3.npy', pdi[psids])

# %% [code]


# %% [code]
